### Apellidos y Nombres:

Lettere Dragosavljevich Mathias Giuseppe

### Fecha:

12-10-2023

# **Preprocesamiento de datos con Pyspark**


## Google Colab Setup

If you are going to use Google Colab instead of a Spark Cluster, you will need to run the following code to install Apache Spark.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
#If the following links don't work, you will have to update them with the last versions of Apache Spark
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

## Setup


In [4]:
# Installing required packages
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=da36567aa315dd190bb6342cfe86107e56e309b4883986ab56129206fcf5fd9d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [5]:
import findspark
findspark.init()

In [6]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

#### Creating the spark session and context


In [7]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#### Initialize Spark session



In [8]:
spark

## Exercise 2 - Load the data and Spark dataframe


## Load the dataset into your Colab directory from your local system


In [9]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [12]:
dfsFlight = spark.read.csv("flights-larger.csv", header=True, inferSchema=True, nullValue= 'NA')
print(dfsFlight.printSchema())

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: integer (nullable = true)

None


## Preprocesamiento




In [13]:
from pyspark.sql.functions import col, isnan, when, count, isnull, max, min, mode, lit

In [14]:
res = [count(when((col(c) == ' ')|( col(c).isNull()), c)).alias(c) for c in dfsFlight.columns]
dfsFlight.select(*res).show()
#def check_for_null_or_nan(df):
    #null_or_nan = lambda x: isnan(x) | isnull(x)
    #func = lambda x: df.filter(null_or_nan(x)).count()
   # print(*[f'{i} has {func(i)} nans/nulls' for i in df.columns if func(i)!=0],sep='\n')

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
|  0|  0|  0|      0|     0|  0|   0|     0|       0|16711|
+---+---+---+-------+------+---+----+------+--------+-----+



In [15]:
# Reemplazar nulos con la moda
dfsTemp = dfsFlight
modDel = dfsTemp.agg(mode('delay')).collect()[0][0]


dfsClean = dfsTemp.fillna({'delay': modDel})

In [16]:
res = [count(when((col(c) == ' ')|( col(c).isNull()), c)).alias(c) for c in dfsClean.columns]
dfsClean.select(*res).show()

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
|  0|  0|  0|      0|     0|  0|   0|     0|       0|    0|
+---+---+---+-------+------+---+----+------+--------+-----+



### Miles a KM


In [17]:
dfsClean = dfsClean.withColumn("km", col("mile") * lit(1.60934))

dfsClean = dfsClean.drop("mile")

dfsClean.show()

+---+---+---+-------+------+---+------+--------+-----+----------+
|mon|dom|dow|carrier|flight|org|depart|duration|delay|        km|
+---+---+---+-------+------+---+------+--------+-----+----------+
| 10| 10|  1|     OO|  5836|ORD|  8.18|      51|   27| 252.66638|
|  1|  4|  1|     OO|  5866|ORD|  15.5|     102|   -7| 749.95244|
| 11| 22|  1|     OO|  6016|ORD|  7.17|     127|  -19|1187.69292|
|  2| 14|  5|     B6|   199|JFK| 21.17|     365|   60|3617.79632|
|  5| 25|  3|     WN|  1675|SJC| 12.92|      85|   22| 621.20524|
|  3| 28|  1|     B6|   377|LGA| 13.33|     182|   70|1731.64984|
|  5| 28|  6|     B6|   904|ORD|  9.58|     130|   47| 1190.9116|
|  1| 19|  2|     UA|   820|SFO| 12.75|     123|  135|1092.74186|
|  8|  5|  5|     US|  2175|LGA|  13.0|      71|  -10| 344.39876|
|  5| 27|  5|     AA|  1240|ORD| 14.42|     195|  -11|1926.37998|
|  8| 20|  6|     B6|   119|JFK| 14.67|     198|   20|1902.23988|
|  2|  3|  1|     AA|  1881|JFK| 15.92|     200|   -9| 1754.1806|
|  8| 26| 

## Indexación



In [18]:
from pyspark.ml.feature import StringIndexer

In [19]:
# “carrier_idx” y “org_idx”
indexer = StringIndexer(inputCols=['carrier', 'org'],
                        outputCols=['carrier_idx', 'org_idx'])

## Bucketing de la variable departure time

In [20]:
from pyspark.ml.feature import Bucketizer

In [21]:
buckets = Bucketizer(splits=[0,3,6,9,12,15,18,21,24],
                     inputCol="depart",
                     outputCol="depart_bucket")

## One-Hot Encoding y Consolidacion

In [22]:
from pyspark.ml.feature import OneHotEncoder

In [23]:
onehot = OneHotEncoder(inputCols=['depart_bucket', 'org_idx', 'dow', 'mon'],
                          outputCols=['depart_dummy', 'org_dummy', 'dow_dummy', 'mon_dummy'])

## Consolidar columnas (features)

In [24]:
from pyspark.ml.feature import VectorAssembler

In [54]:
assembler = VectorAssembler(inputCols=['km'],
                            outputCol='features')


In [71]:
dfsClean = assembler.transform(dfsClean)

## Entrenamiento y Prueba


In [74]:
flyTrain, flyTest = dfsClean.randomSplit([0.8, 0.2], seed=23)

[flyTest.count(), flyTrain.count()]

[55438, 219562]

## Regresion Lineal


In [27]:
from pyspark.ml.regression import LinearRegression


In [28]:
regr = LinearRegression(labelCol="duration")

## Metricas de Evaluacion



In [29]:
from pyspark.ml.evaluation import RegressionEvaluator

In [30]:
evaluator1 = RegressionEvaluator(labelCol='duration')
evaluator2 = RegressionEvaluator(labelCol='duration', metricName='r2')

## Pipeline

In [55]:
from pyspark.ml import Pipeline

In [56]:
pipeline = Pipeline(stages=[indexer, buckets, onehot, assembler, regr])


In [57]:
pipeline = pipeline.fit(flyTrain)


In [58]:
predictions = pipeline.transform(flyTrain)
predictions['duration', 'prediction'].show()

+--------+------------------+
|duration|        prediction|
+--------+------------------+
|     420| 496.6655026793104|
|     300| 347.3164513637969|
|     315| 359.5082106548592|
|     320| 359.5082106548592|
|     386| 359.5082106548592|
|     375|   345.97535784178|
|     379|   345.97535784178|
|     325| 373.8944866183127|
|     310| 359.3862930619486|
|     180|190.16467410200343|
|     170|133.59491099147425|
|     165|133.59491099147425|
|     210|222.71667140913985|
|     165|133.59491099147425|
|      70| 75.68405435892817|
|     115|133.59491099147425|
|     160|133.59491099147425|
|     164|133.59491099147425|
|     130|133.59491099147425|
|     130|133.59491099147425|
+--------+------------------+
only showing top 20 rows



In [59]:
# Intercept del modelo
print(pipeline.stages[4].intercept)

44.229315387987356


In [60]:
#Coeficientes
print(pipeline.stages[4].coefficients)

[0.07575626835263108]


## Cross Validation

In [78]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [77]:
params = ParamGridBuilder().build()

In [75]:
cv1 = CrossValidator(estimator=regr,
                    estimatorParamMaps=params,
                    evaluator=evaluator1,
                    numFolds=5, seed=13)

In [76]:
cv2 = CrossValidator(estimator=regr,
                    estimatorParamMaps=params,
                    evaluator=evaluator2,
                    numFolds=5, seed=13)

### Para RMSE

In [79]:
cv1 = cv1.fit(flyTest)

In [80]:
cv1.avgMetrics

[17.29903479403332]

### Para R^2

In [81]:
cv2 = cv2.fit(flyTrain)

In [82]:
cv2.avgMetrics

[0.961126792585574]

In [83]:
spark.stop()